In [ ]:
import os
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript

# ─── Helper to hit sec‑api.io full‑text search ────────────────
def full_text_search(api_key, query, form_types, start_date, end_date, page=1):
    url = f"https://api.sec-api.io/full-text-search?token={api_key}"
    headers = {
        "Authorization": api_key,
        "Content-Type":  "application/json",
        "Accept":        "application/json",
    }
    payload = {
        "query":     query,
        "formTypes": form_types,
        "startDate": start_date,
        "endDate":   end_date,
        "page":      str(page),
    }
    resp = requests.post(url, json=payload, headers=headers, timeout=15)
    resp.raise_for_status()
    return resp.json()

# ─── CONFIG ──────────────────────────────────────────────────────────────
API_KEY = os.getenv("SEC_API_KEY", "ca91a88f0a033a9774da5fba074cb8660f145f5ce5f3eb8eac83973c458f7923")
if not API_KEY or API_KEY.startswith("<"):
    raise RuntimeError("🔑 Please set a valid SEC_API_KEY env var")

FORM_TYPES = ["424B2"]
START_DT    = "2020-01-01"
END_DT      = "2026-06-30"

# ─── load your CUSIP list from file ─────────────────────────────────────────
cusip_list = []
cusip_file = "cusiplist.txt"
if not os.path.exists(cusip_file):
    raise FileNotFoundError(f"Could not find `{cusip_file}` in working directory")
with open(cusip_file, "r") as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith("#"):
            cusip_list.append(line)

# ─── BUILD UI ───────────────────────────────────────────────────────────────
cusip_dd  = widgets.Dropdown(options=cusip_list, description="CUSIP:")
btn       = widgets.Button(description="Fetch & Open Prospectus", button_style="primary")
output    = widgets.Output()
ui        = widgets.VBox([cusip_dd, btn, output])
display(ui)

# ─── CALLBACK ──────────────────────────────────────────────────────────────
def on_click_fetch_open(_):
    with output:
        clear_output()
        cusip = cusip_dd.value
        print(f"🔎 Looking up 424B2 for {cusip}…", end=" ")

        try:
            data = full_text_search(
                API_KEY,
                f'"{cusip}"',
                FORM_TYPES,
                START_DT, END_DT,
                page=1
            )
        except Exception as e:
            print("❌ Search error:", e)
            return

        filings = data.get("filings") or data.get("data", {}).get("filings", [])
        if not filings:
            print("ℹ️ No 424B2 filings found.")
            return

        # pick the most recent by filedAt/filingDate
        best = max(filings, key=lambda f: f.get("filedAt") or f.get("filingDate",""))
        link = best.get("linkToFilingDetails") or best.get("filingUrl") or best.get("url")
        if not link:
            print("❌ No prospectus URL found in that filing.")
            return

        print("✅ 1. right click, 2. print, 3. destination: save as PDF")
        # open a new mini‑window directly to the SEC page
        js = f"""
        window.open(
          "{link}",
          "_blank",
          "toolbar=no,location=no,status=no,menubar=no,scrollbars=yes,resizable=yes,width=1200,height=1000"
        );
        """
        display(Javascript(js))

btn.on_click(on_click_fetch_open)
